In [31]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

Preparing match DF

In [3]:
path = 'data/database.sqlite'

In [4]:
connection = sqlite3.connect(path)

In [43]:
query = '''
  SELECT * 
  FROM Match
'''
match_df = pd.read_sql_query(query, connection)
match_df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
format_data = "%Y-%m-%d %H:%M:%S"
match_df['date'] = match_df['date'].apply(lambda x: datetime.strptime(x, format_data))


0       2008-08-17
1       2008-08-16
2       2008-08-16
3       2008-08-17
4       2008-08-16
           ...    
25974   2015-09-22
25975   2015-09-23
25976   2015-09-23
25977   2015-09-22
25978   2015-09-23
Name: date, Length: 25979, dtype: datetime64[ns]

In [48]:
identification_cols = ['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'goal', 'shoton', 'shotoff', 'possession']
betting_cols = list(match_df.columns[-30:]) 
cols = identification_cols + betting_cols
match_df = match_df[cols]

In [49]:
for res in ['H', 'D', 'A']:
    match_df[f'mean_bet_{res}'] = match_df[[col for col in betting_cols if col[-1] == res]].mean(axis=1)

C:\Users\EB\AppData\Local\Temp/ipykernel_1188/3007036654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df[f'mean_bet_{res}'] = match_df[[col for col in betting_cols if col[-1] == res]].mean(axis=1)


In [50]:
match_df = match_df[identification_cols + [f'mean_bet_{res}' for res in ['H', 'D', 'A']]]

In [51]:
print(list(match_df.columns))

['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'goal', 'shoton', 'shotoff', 'possession', 'mean_bet_H', 'mean_bet_D', 'mean_bet_A']


A df with all the wins, we can use this as part of x to describe how good a team is in a given season


In [52]:
wins = pd.read_sql_query("""
  SELECT season, home_team_api_id, home_team AS team, SUM(home_win + away_win) as wins 
  FROM
    (
    SELECT MATCH.id, MATCH.home_team_api_id, Match.season, t1.team_long_name as home_team, t2.team_long_name as away_team,
      (CASE WHEN home_team_goal > away_team_goal THEN 1 ELSE 0 END) AS home_win,
      (CASE WHEN home_team_goal = away_team_goal THEN 1 ELSE 0 END) AS draw,
      (CASE WHEN home_team_goal < away_team_goal THEN 1 ELSE 0 END) AS away_win
    FROM MATCH 
    JOIN TEAM as t1 ON t1.team_api_id = MATCH.home_team_api_id 
    JOIN TEAM as t2 ON t2.team_api_id = MATCH.away_team_api_id
    WHERE league_id = 1729)
    GROUP BY home_team, season
    ORDER BY season DESC, wins DESC
  """, connection)

In [53]:
wins

,season,home_team_api_id,team,wins
0,2015/2016,8456,Manchester City,17
1,2015/2016,8466,Southampton,16
2,2015/2016,9826,Crystal Palace,16
3,2015/2016,10194,Stoke City,15
4,2015/2016,9825,Arsenal,15
...,...,...,...,...
155,2008/2009,10261,Newcastle United,12
156,2008/2009,8650,Liverpool,12
157,2008/2009,8655,Blackburn Rovers,12
158,2008/2009,8549,Middlesbrough,10


In [54]:
team = pd.read_sql_query("""SELECT * FROM TEAM""", connection)

In [55]:
team

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


In [57]:
df = match_df[match_df['league_id'] == 1729].copy().reset_index()
df['goal_diff'] = df['home_team_goal'] - df['away_team_goal']
df = df[['match_api_id', 'season', 'date', 'home_team_api_id', 'away_team_api_id', 'goal_diff', 'mean_bet_H', 'mean_bet_D', 'mean_bet_A']]
df

,match_api_id,season,date,home_team_api_id,away_team_api_id,goal_diff,mean_bet_H,mean_bet_D,mean_bet_A
0,489042,2008/2009,2008-08-17,10260,10261,0,1.278889,4.822222,10.083333
1,489043,2008/2009,2008-08-16,9825,8659,1,1.208889,5.522222,12.222222
2,489044,2008/2009,2008-08-16,8472,8650,-1,4.925556,3.450000,1.671111
3,489045,2008/2009,2008-08-16,8654,8528,1,1.874444,3.283333,3.938889
4,489046,2008/2009,2008-08-17,10252,8456,2,1.914444,3.277778,3.814444
...,...,...,...,...,...,...,...,...,...
3035,1988795,2015/2016,2015-10-17,8466,8197,0,1.765714,3.711429,4.814286
3036,1988796,2015/2016,2015-10-19,10003,10194,-1,2.047143,3.315714,3.958571
3037,1988797,2015/2016,2015-10-17,8586,8650,0,2.475714,3.377143,2.938571
3038,1988798,2015/2016,2015-10-17,9817,9825,-3,5.791429,4.044286,1.607143


In [58]:
df['date'][0]

Timestamp('2008-08-17 00:00:00')

In [62]:
groups = df.set_index('date').groupby('home_team_api_id')
for i, group in groups:
    print(i)
    print(group.sort_index())
    for season, season_df in gr
    next_game = data.iloc[1:]
    prev_game = groups.copy()


8191
            match_api_id     season  home_team_api_id  away_team_api_id  \
date                                                                      
2009-08-19        658589  2009/2010              8191             10260   
2009-08-23        658601  2009/2010              8191              8668   
2009-09-19        658631  2009/2010              8191              8472   
2009-10-03        658684  2009/2010              8191              8658   
2009-10-24        658699  2009/2010              8191              8528   
2009-10-31        658713  2009/2010              8191              8667   
2009-11-21        658792  2009/2010              8191             10252   
2009-12-12        658826  2009/2010              8191              9879   
2009-12-16        658834  2009/2010              8191              9825   
2009-12-26        658923  2009/2010              8191              8559   
2010-01-30        658973  2009/2010              8191              8455   
2010-02-06        65